In [1]:
from builtins import *
import keras
import pandas as pd
import numpy as np
import math
from collections import Counter
import tensorflow as tf

Using TensorFlow backend.
D:\anaconda\anaconda3\envs\tf2\lib\site-packages\tensorflow\python\framework\dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
D:\anaconda\anaconda3\envs\tf2\lib\site-packages\tensorflow\python\framework\dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
D:\anaconda\anaconda3\envs\tf2\lib\site-packages\tensorflow\python\framework\dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
D:\anaconda\anaconda3\envs\tf2\lib\site-packages\tensorflow\py

In [2]:
class Dataloader():
    def __init__(self,data_path):
        self.data_path = data_path

    def load_adj(self):
        df_adj = pd.read_csv(open(self.data_path + '/adj.tsv'), sep='\t', dtype={0:np.int32, 1:np.int32})
        return df_adj

    def load_latest_session(self):
        ret = []
        for line in open(self.data_path + '/latest_sessions.txt'):
            chunks = line.strip().split(',')
            ret.append(chunks)
        return ret

    def load_map(self, name='user'):
        if name == 'user':
            file_path = self.data_path + '/user_id_map.tsv'
        elif name == 'item':
            file_path = self.data_path + '/item_id_map.tsv'
        else:
            raise NotImplementedError
        id_map = {}
        for line in open(file_path):
            k, v = line.strip().split('\t')
            id_map[k] = str(v)
        return id_map

    def load_data(self):
        adj = self.load_adj()
        latest_sessions = self.load_latest_session()
        user_id_map = self.load_map('user')
        item_id_map = self.load_map('item')
        train = pd.read_csv(open(self.data_path + '/train.tsv'), sep='\t', dtype={0: np.int32, 1: np.int32, 3: np.float32})
        valid = pd.read_csv(open(self.data_path + '/valid.tsv'), sep='\t', dtype={0: np.int32, 1: np.int32, 3: np.float32})
        test = pd.read_csv(open(self.data_path + '/test.tsv'), sep='\t', dtype={0: np.int32, 1: np.int32, 3: np.float32})
        return [adj, latest_sessions, user_id_map, item_id_map, train, valid, test]

In [3]:
data_path = 'H:/研究生阶段/研二上/RecommenderSystems/socialRec/data/data'
data = Dataloader(data_path).load_data()

In [4]:
adj_info = data[0]
latest_per_user_by_time = data[1]
user_id_map = data[2]
item_id_map = data[3]
train_df = data[4]
valid_df = data[5]
test_df = data[6]
data_ = [train_df,valid_df,test_df]
all_data = pd.concat(data_)

In [5]:
class construct_graph():
    def __init__(self,datadf,num_nodes,max_degree,adj_info,latest_peruser_time):
        self.datadf = datadf
        self.num_nodes = num_nodes
        self.max_degree = max_degree
        self.adj_info = adj_info
        self.latest_peruser_time = latest_peruser_time
        self.visible_time = self.user_visiable_time()
        self.adj,self.deg = self.construct_adj()

    def construct_adj(self):
        '''
        Construct adj table used during training/validing/testing.
        '''
        adj = self.num_nodes*np.ones((self.num_nodes+1, self.max_degree), dtype=np.int32)
        deg = np.zeros((self.num_nodes,))
        missed = 0
        for nodeid in self.datadf.UserId.unique():
            neighbors = np.array([neighbor for neighbor in
                             self.adj_info.loc[self.adj_info['Follower']==nodeid].Followee.unique()], dtype=np.int32)
            deg[nodeid] = len(neighbors)
            if len(neighbors) == 0:
                missed += 1
                continue
            if len(neighbors) > self.max_degree:
                neighbors = np.random.choice(neighbors, self.max_degree, replace=False)
            elif len(neighbors) < self.max_degree:
                lentmp = self.max_degree - len(neighbors)
                neighborstmp = np.random.choice(neighbors,lentmp,replace=True)
                neighbors = np.concatenate((neighbors,neighborstmp))
            adj[nodeid, :] = neighbors
        return adj, deg

    def user_visiable_time(self):
        visible_time = []
        for l in self.latest_peruser_time:
            timeid = max(loc for loc, val in enumerate(l) if val == 'NULL') + 1
            visible_time.append(timeid)
            assert timeid > 0 and timeid < len(l), 'Wrong when create visible time {}'.format(timeid)
        return visible_time

    def remove_infoless(self):
        data = self.datadf.loc[self.deg[self.datadf['UserId']]!=0]
        saved_session_ids = []
        for sessid in data.SessionId.unique():
            userid,timeid = sessid.split('_')
            userid,timeid = int(userid),int(timeid)
            count1 = 0
            for neightbor in self.adj[userid, :]:
                if self.visible_time[neightbor] <= timeid and self.deg[neightbor] > 0:
                    count1 += 1

            if count1 < self.max_degree and count1>0:
                saved_session_ids.append(sessid)
        return saved_session_ids

    def make_layer_1(self,num_samples_1,nodeids,timeids):
        adj_lists_1 = []
        num_samples_1 = num_samples_1
        nodeids = nodeids
        timeids = timeids
        for idx in range(len(nodeids)):
            node = nodeids[idx]
            timeid = timeids[idx]
            adj = self.adj[node,:]
            neighbors = []
            for neighbor in adj:
                if self.visible_time[neighbor] <= timeid and self.deg[neighbor] > 0:
                    # for second_neighbor in self.adj[neighbor]:
                    # 	if self.visible_time[second_neighbor] <= timeid:
                    # 		for second_neighbor in self.adj[neighbor]:
                    # 			if self.visible_time[second_neighbor] <= timeid:
                    # 				neighbors.append(neighbor)
                    # 				break
                    neighbors.append(neighbor)

            assert len(neighbors) > 0
            if len(neighbors) < num_samples_1:
                neighbors = np.random.choice(neighbors, num_samples_1, replace=True)
            elif len(neighbors) >= num_samples_1:
                neighbors = np.random.choice(neighbors, num_samples_1, replace=False)
            adj_lists_1.append(neighbors)
        return np.array(adj_lists_1,dtype = np.int32)

    def support_node(self):
        self.sessionids = self.remove_infoless()
        nodeids = [int(sessionid.split('_')[0]) for sessionid in self.sessionids]
        timeids = [int(sessionid.split('_')[1]) for sessionid in self.sessionids]
        self.layer1 = self.make_layer_1(num_samples_1=10,nodeids=nodeids,timeids=timeids)
        support_layer1 = []
        idx = 0
        for sessionid in self.sessionids:
            # userid = int(sessionid.split('_')[0])
            timeid = int(sessionid.split('_')[1])
            layer1 = self.layer1[idx]
            idx+=1
            per_layer = []
            for support_node in layer1:
                support_session_id = str(self.latest_peruser_time[support_node][timeid])
                per_layer.append(support_session_id)
            support_layer1.append(np.array(per_layer))
        return np.array(support_layer1),self.sessionids,self.layer1

In [6]:
train_layer1,train_sessionids,layer1 = construct_graph(datadf=train_df,num_nodes=len(user_id_map),
                        max_degree=50,adj_info=adj_info,
                        latest_peruser_time=latest_per_user_by_time).support_node()


In [7]:
train_sessionids = np.array(train_sessionids)
train_sessionids.shape

(155656,)

In [8]:
class sessionpadding():
    '''
    different sessions have different lengths, model needs same length for next thing;
    use keras to padding items in every session.
    datadf:dataloader.py data[4],data[5],data[6];
    '''

    def __init__(self,datadf,maxlen=20):
        self.datadf = datadf
        self.maxlen = maxlen

    def padding(self):
        self.padding_dict = self.datadf.sort_values(by=['TimeId']).groupby('SessionId')['ItemId'].apply(list).to_dict()
        list_v = [i for i in self.padding_dict.values()]
        list_k = [i for i in self.padding_dict.keys()]
        x_data = []
        y_data = []
        for i in list_v:
            x = i[1:]
            y = i[:-1]
            x_data.append(x)
            y_data.append(y)
        # use -1 padding values
        x_list_v_afterpadding = tf.keras.preprocessing.sequence.pad_sequences(sequences=x_data,maxlen=self.maxlen,dtype='int32',padding='post',truncating='post',value=0).tolist()
        y_list_v_afterpadding = tf.keras.preprocessing.sequence.pad_sequences(sequences=y_data,maxlen=self.maxlen,dtype='int32',padding='post',truncating='pre',value=0).tolist()
        self.padding_x = dict(zip(list_k,x_list_v_afterpadding))
        self.padding_y = dict(zip(list_k,y_list_v_afterpadding))
        return self.padding_x,self.padding_y

In [9]:
train_padding_x,train_padding_y = sessionpadding(train_df).padding()

In [10]:
#train_sessionids have 155658 user sessionid ,trian_sessionids are array
#train_layer1 have 155658 layer,each one has 10 friends sessionids,they are array
#next,i should turn sessionids to padded items.
#(nums,1)---->(nums,20)
#(nums,10)--->(nums,10,20)
#next step is making itemids to embedding itemids
#(nums,20,100)
#(nums,10,20,100)

In [11]:
train_input_x_user_data = []
train_input_y_user_data = []
for sessionid in train_sessionids:
    train_input_x_user_data.append(train_padding_x[sessionid])
    train_input_y_user_data.append(train_padding_y[sessionid])
train_input_x_friends_data = []
train_input_x_friends_iddata = []
delete_idx = []
for nodeidx in range(len(train_layer1)):
    node = train_layer1[nodeidx]
    train_input_x_friends_iddata.append(node)
    user_friend_item = []
    idx = 0
    for idx in range(len(node)):
        sessionid = node[idx]
        if sessionid not in train_padding_x.keys():
            delete_idx.append(nodeidx)
            for i in range(10):
                user_friend_item.append([1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20])
            break
        user_friend_item.append(train_padding_x[sessionid])
    train_input_x_friends_data.append(user_friend_item)
    
train_input_x_user_data = np.array(train_input_x_user_data)
train_input_y_user_data = np.array(train_input_y_user_data)
train_input_x_friends_data = np.array(train_input_x_friends_data)
train_input_x_friends_iddata = np.array(train_input_x_friends_iddata)
train_input_x_user_data = np.delete(train_input_x_user_data, delete_idx, 0)
train_input_y_user_data = np.delete(train_input_y_user_data, delete_idx, 0)
train_input_x_friends_data = np.delete(train_input_x_friends_data, delete_idx, 0)
train_input_x_friends_iddata = np.delete(train_input_x_friends_iddata, delete_idx, 0)
print(train_input_x_user_data.shape,train_input_y_user_data.shape,train_input_x_friends_data.shape,train_input_x_friends_iddata.shape)


(155659, 20) (155659, 20) (155659, 10, 20) (155659, 10)


In [12]:
train_input_x_friends_user_iddata = []
for i in range(len(train_input_x_friends_iddata)):
    sessionids = train_input_x_friends_iddata[i]
    friendsids = []
    for sessionid in sessionids:
        
        friendsid = int(sessionid.split('_')[0])
        friendsids.append(friendsid)
    train_input_x_friends_user_iddata.append(friendsids)
train_input_x_friends_user_iddata = np.array(train_input_x_friends_user_iddata)
print(train_input_x_friends_user_iddata.shape,train_input_x_friends_user_iddata[23123])

(155659, 10) [ 2122 10593   889   889   889 10593  3311 10593  2122  3311]


In [13]:
train_input_x_friends_data_0 = train_input_x_friends_data[:,0,:]
train_input_x_friends_user_iddata_0 = train_input_x_friends_user_iddata[:,0]
train_input_x_friends_user_iddata.shape

(155659, 10)

In [174]:
item_num = len(item_id_map)
user_num = len(user_id_map)
model = keras.Model()
Input_user = keras.Input(shape=(20,))
Input_user_friends = keras.Input(shape=(10,20,))
Input_user_friends_0 = keras.Input(shape=(20,))
Input_user_friends_0_u = keras.Input(shape=(1,))
item_embedding_layer = keras.layers.Embedding(item_num+1,50,mask_zero=True)
user_embedding_layer = keras.layers.Embedding(user_num+1,50)
LSTM_friends = keras.layers.LSTM(units=100,activation='tanh',recurrent_activation='sigmoid',use_bias=True,
                 bias_initializer='zero',
                  return_sequences=False)

user_embedding_friend_0=user_embedding_layer(Input_user_friends_0_u)
user_embedding_friend_0 = keras.layers.Reshape((50,))(user_embedding_friend_0)
item_embedding_friend_0=item_embedding_layer(Input_user_friends_0)
LSTM_friend_0 = LSTM_friends(item_embedding_friend_0)
LSTM_friend_0 = keras.layers.Dense(50, activation='relu')(LSTM_friend_0)
friend_0 = keras.layers.concatenate([user_embedding_friend_0,LSTM_friend_0])


item_embedding=item_embedding_layer(Input_user)
LSTM = keras.layers.LSTM(units=100,activation='tanh',recurrent_activation='sigmoid',use_bias=True,
                 bias_initializer='zero',
                  return_sequences=False)(item_embedding)
output = keras.layers.concatenate([LSTM,friend_0])
y = keras.layers.Dense(20, activation='softmax')(output)
model = keras.Model([Input_user,Input_user_friends_0,Input_user_friends_0_u], y)
opt = keras.optimizers.Adam(lr=1e-3, decay=1e-3 / 200)
model.compile(loss="mean_absolute_percentage_error", optimizer=opt)

In [68]:
model.fit([train_input_x_user_data,train_input_x_friends_data_0], train_input_y_user_data,
epochs=1, batch_size=8)


Epoch 1/1
155600/155600 [==============================] - 981s 6ms/step - loss: 133801.48490s - loss: 133870.30 - ETA: 0s - l


In [37]:
train_layer1[132]

array(['1161_329', '1161_329', '1161_329', '1161_329', '1161_329',
       '1161_329', '1161_329', '1161_329', '1161_329', '1161_329'],
      dtype='<U8')

In [43]:
train_input_x_user_data[345]

array([1447, 1277, 2793, 2794,  856, 1395, 1564, 2792,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0])

model.summary()

In [74]:
model.summary()

Model: "model_8"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_17 (InputLayer)           (None, 20)           0                                            
__________________________________________________________________________________________________
input_19 (InputLayer)           (None, 20)           0                                            
__________________________________________________________________________________________________
embedding_9 (Embedding)         (None, 20, 50)       629600      input_19[0][0]                   
                                                                 input_17[0][0]                   
__________________________________________________________________________________________________
input_20 (InputLayer)           (None, 1)            0                                      

In [ ]:
#embedding
input_data_item = keras.Input(shape=(1,),batch_shape=(2968,1))
item_embedding = keras.layers.Embedding(2968, 50, input_length=1)(input_data_item)
ids=[[112,4,5,1],[112,4,5,1],[112,4,5,1],[112,4,5,1]]
iembs = tf.nn.embedding_lookup(item_embedding,ids)
print(iembs)
model = keras.models.Model(input_data_item,item_embedding)
input_data_item=np.arange(2967)
input_data_item = np.array(input_data_item)
output = model_item.predict(input_data_item)
#user_embedding
len(data[2])
input_data_user=np.arange(10558)
input_data_user = np.array(input_data_user)
model = keras.models.Sequential()
model.add(keras.layers.Embedding(10559,50,input_length=1))
model.compile('rmsprop','mse')
output = model.predict(input_data_user)
user_embedding = output
input_data_user
#item_embedding
len(data[3])
input_data_item=np.arange(2967)
input_data_item = np.array(input_data_item)
model_item = keras.models.Sequential()
model_item.add(keras.layers.Embedding(2968,50,input_length=1))
model_item.compile('rmsprop','mse')
output = model_item.predict([1,2,3])
item_embedding = output
item_embedding.shape
#dense,global
inputs = keras.Input(shape=(1,50))
x = keras.layers.Dense(100, activation='relu')(inputs)
model_dense = keras.Model(inputs=inputs, outputs=x)
output = model.predict(user_embedding)